<a href="https://colab.research.google.com/github/triantonugroho/Applied-Deep-Learning-Task/blob/main/EndTermTask/Generated%20Images/MNIST/EndTermTask_(DCGAN_MNIST).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NAME : TRIANTO HARYO NUGROHO**

**NPM : 2306288931**

##1. Import Library

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Reshape
from tensorflow.keras.layers import Activation, BatchNormalization, UpSampling2D, Dense, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math

##2. Generator Model Function

In [ ]:
def generator_model():
  model = Sequential()
  model.add(Dense(units=1024, input_dim=100))
  model.add(Activation('tanh'))
  model.add(Dense(128*7*7))
  model.add(BatchNormalization())
  model.add(Activation('tanh'))
  model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
  model.add(UpSampling2D(size=(2, 2)))
  model.add(Conv2D(64, (5, 5), padding='same'))
  model.add(Activation('tanh'))
  model.add(UpSampling2D(size=(2, 2)))
  model.add(Conv2D(1, (5, 5), padding='same'))
  model.add(Activation('tanh'))
  return model

##3. Discriminator Model Function

In [ ]:
def discriminator_model():
  model = Sequential()
  model.add(Conv2D(64, (5, 5), padding='same', input_shape=(28, 28, 1)))
  model.add(Activation('tanh'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(128, (5, 5)))
  model.add(Activation('tanh'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('tanh'))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))
  return model

##4. Generator Containing Discriminator Function

In [ ]:
def generator_containing_discriminator(g, d):
  model = Sequential()
  model.add(g)
  d.trainable = False
  model.add(d)
  return model

##5. Combines Image Function

In [ ]:
def combine_images(generated_images):
  num = generated_images.shape[0]
  width = int(math.sqrt(num))
  height = int(math.ceil(float(num)/width))
  shape = generated_images.shape[1:3]
  image = np.zeros((height*shape[0], width*shape[1]), dtype=generated_images.dtype)
  for index, img in enumerate(generated_images):
    i = int(index/width)
    j = index % width
    image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]]= img[:, :, 0]
  return image


##6. Training Function

In [ ]:
def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

    # Create discriminator and generator models
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)

    # Optimizers for generator and discriminator
    d_optim = SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)

    g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)

    # Training loop
    for epoch in range(20):
        print(f"Epoch is {epoch}")
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            # Generate noise for the generator
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))

            # Get a batch of real images
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]

            # Generate images from the noise
            generated_images = g.predict(noise, verbose=0)

            # Save generated images every 20 batches
            if index % 20 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(f"{epoch}_{index}.png")

            # Train discriminator
            X = np.concatenate((image_batch, generated_images))
            y = np.concatenate((np.ones(BATCH_SIZE), np.zeros(BATCH_SIZE)))
            d_loss = d.train_on_batch(X, y)
            print(f"Batch {index} d_loss: {d_loss}")

            # Train generator
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, np.ones(BATCH_SIZE))
            d.trainable = True
            print(f"Batch {index} g_loss: {g_loss}")

            # Save weights of generator and discriminator every 10 batches
            if index % 10 == 9:
                g.save_weights('generator.weights.h5')
                d.save_weights('discriminator.weights.h5')


##7. Image Generation Function

In [ ]:
def generate(BATCH_SIZE, nice=False):
  g = generator_model()
  g.compile(loss='binary_crossentropy', optimizer="SGD")
  g.load_weights('generator')
  if nice:
    d = discriminator_model()
    d.compile(loss='binary_crossentropy', optimizer="SGD")
    d.load_weights('discriminator')
    noise = np.uniform(-1, 1, (BATCH_SIZE*20, 100))
    generated_images = g.predict(noise, verbose=1)
    d_pred = d.predict(generated_images, verbose=1)
    index = np.arange(0, BATCH_SIZE*20)
    index.resize((BATCH_SIZE*20, 1))
    pre_with_index = list(np.append(d_pred, index, axis=1))
    pre_with_index.sort(key=lambda x: x[0], reverse=True)
    nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
    nice_images = nice_images[:, :, :, None]
    for i in range(BATCH_SIZE):
      idx = int(pre_with_index[1][1])
      nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
    image = combine_images(nice_images)
  else:
    noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
    generated_images = g.predict(noise, verbose=1)
    image = combine_images(generated_images)
  image = image*127.5+127.5
  Image.fromarray(image.astype(np.uint8)).save("generated_image.png")

In [ ]:
pip install tensorflow keras numpy pillow


In [ ]:
# Set BATCH_SIZE and EPOCHS
BATCH_SIZE = 128
EPOCHS = 100

# Train the GAN
train(BATCH_SIZE)

# Generate images after training
generate(BATCH_SIZE, nice=False)  # For raw generated images
generate(BATCH_SIZE, nice=True)   # For discriminator-selected images


Epoch is 0
Batch 0 d_loss: 0.6954244375228882
Batch 0 g_loss: [array(0.69542444, dtype=float32), array(0.69542444, dtype=float32), array(0.69542444, dtype=float32)]
Batch 1 d_loss: 0.6885690689086914
Batch 1 g_loss: [array(0.68856907, dtype=float32), array(0.68856907, dtype=float32), array(0.68856907, dtype=float32)]


Batch 2 d_loss: 0.6836385726928711


Batch 2 g_loss: [array(0.6836386, dtype=float32), array(0.6836386, dtype=float32), array(0.6836386, dtype=float32)]
Batch 3 d_loss: 0.6787784695625305
Batch 3 g_loss: [array(0.67877847, dtype=float32), array(0.67877847, dtype=float32), array(0.67877847, dtype=float32)]
Batch 4 d_loss: 0.6717756390571594
Batch 4 g_loss: [array(0.67177564, dtype=float32), array(0.67177564, dtype=float32), array(0.67177564, dtype=float32)]
Batch 5 d_loss: 0.6640976071357727
Batch 5 g_loss: [array(0.6640976, dtype=float32), array(0.6640976, dtype=float32), array(0.6640976, dtype=float32)]
Batch 6 d_loss: 0.6556447744369507
Batch 6 g_loss: [array(0.6556448, dtype=float32), array(0.6556448, dtype=float32), array(0.6556448, dtype=float32)]
Batch 7 d_loss: 0.6477659344673157
Batch 7 g_loss: [array(0.64776593, dtype=float32), array(0.64776593, dtype=float32), array(0.64776593, dtype=float32)]
Batch 8 d_loss: 0.6398808360099792
Batch 8 g_loss: [array(0.63988084, dtype=float32), array(0.63988084, dtype=float32), 